# Cài đặt thư viện

In [1]:
# Cài đặt các thư viện cần thiết
!uv pip install transformers torch pandas scikit-learn tqdm





                         ./o.
                       ./sssso-
                     `:osssssss+-
                   `:+sssssssssso/.
                 `-/ossssssssssssso/.
               `-/+sssssssssssssssso+:`
             `-:/+sssssssssssssssssso+/.
           `.://osssssssssssssssssssso++-
          .://+ssssssssssssssssssssssso++:
        .:///ossssssssssssssssssssssssso++:
      `:////ssssssssssssssssssssssssssso+++.
    `-////+ssssssssssssssssssssssssssso++++-
     `..-+oosssssssssssssssssssssssso+++++/`
       ./++++++++++++++++++++++++++++++/:.
      `:::::::::::::::::::::::::------``
┌─────────────── Hardware ────────────────┐
 PC: superkid@loihnt-nitroan51545
│ ├󰍛 CPU: AMD Ryzen 5 5600H
│ ├󰍛 GPU: NVIDIA GeForce GTX 1650 Mobile / Max-Q [Discrete]
│ ├󰍛 GPU: AMD Radeon Vega Series / Radeon Vega Mobile Series [Integrated]
└ └󰍛 Memory: 11.20 GiB / 14.97 GiB (75%)
└──────────────────────────────────────────┘

┌─────────────── Software ────────────────┐
 OS: EndeavourOS x86_64


# Import và Cấu hình

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time
from tqdm.auto import tqdm # Tự động chọn thanh tiến trình phù hợp

# ================= CẤU HÌNH =================
INPUT_FILE = '../dataset/articles_clean.csv'  # <--- Đổi tên file nếu file của bạn khác
MODEL_NAME = "vinai/phobert-base"
SAVE_PATH = "phobert_classifier.pth"

# Cấu hình Training
MAX_LEN = 128       # Độ dài tối đa của câu (128 là đủ cho tin tức, max là 256)
BATCH_SIZE = 16     # Giảm xuống 8 nếu bị lỗi "Out of Memory" (OOM)
EPOCHS = 3          # Số vòng học
LEARNING_RATE = 2e-5

# Tùy chọn: Chỉ lấy một phần dữ liệu để test code cho nhanh?
# Set = None nếu muốn train hết 100% dữ liệu (Sẽ rất lâu, khoảng vài tiếng)
# Set = 5000 để chạy thử xem code có lỗi không
SAMPLE_SIZE = None 

# Kiểm tra GPU
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print(f'✅ Đã tìm thấy GPU: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device("cpu")
    print('⚠️ Không thấy GPU, sẽ chạy bằng CPU (Rất chậm!)')

/home/loiancut/workspace/fake-news-detection/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Đã tìm thấy GPU: NVIDIA GeForce GTX 1650


# Load & Chuẩn bị dữ liệu

In [3]:
# 1. Load dữ liệu
print(f"📂 Đang đọc file {INPUT_FILE}...")
try:
    df = pd.read_csv(INPUT_FILE)
    print(f"   Tổng số dòng: {len(df)}")
except FileNotFoundError:
    print(f"❌ Không tìm thấy file {INPUT_FILE}. Hãy kiểm tra lại tên file!")
    
# 2. Tự động tìm tên cột chứa nội dung
possible_text_cols = ['content', 'text', 'news', 'article', 'body']
col_text = next((col for col in possible_text_cols if col in df.columns), None)

# Tự động tìm tên cột nhãn
possible_label_cols = ['label', 'target', 'is_fake']
col_label = next((col for col in possible_label_cols if col in df.columns), None)

if col_text and col_label:
    print(f"✅ Đã xác định cột nội dung: '{col_text}'")
    print(f"✅ Đã xác định cột nhãn: '{col_label}'")
else:
    raise ValueError(f"❌ Không tìm thấy tên cột phù hợp. Cột hiện có: {list(df.columns)}")

# 3. Lấy mẫu (nếu cần)
if SAMPLE_SIZE and SAMPLE_SIZE < len(df):
    print(f"⚠️ Đang lấy mẫu {SAMPLE_SIZE} dòng để test code...")
    df = df.sample(SAMPLE_SIZE, random_state=42)

texts = df[col_text].values
labels = df[col_label].values

print(f"📊 Dữ liệu sẵn sàng: {len(texts)} mẫu.")

📂 Đang đọc file ../dataset/articles_clean.csv...
   Tổng số dòng: 96239
✅ Đã xác định cột nội dung: 'content'
✅ Đã xác định cột nhãn: 'label'
📊 Dữ liệu sẵn sàng: 96239 mẫu.
   Tổng số dòng: 96239
✅ Đã xác định cột nội dung: 'content'
✅ Đã xác định cột nhãn: 'label'
📊 Dữ liệu sẵn sàng: 96239 mẫu.


# Tokenizer

In [4]:
print("🤖 Đang tải Tokenizer từ VinAI...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Hàm mã hóa
def encode_batch(data_texts, max_len=128):
    input_ids = []
    attention_masks = []
    
    print("🚀 Bắt đầu mã hóa (Tokenizing)...")
    for text in tqdm(data_texts):
        encoded = tokenizer.encode_plus(
            str(text),
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Thực hiện mã hóa
input_ids, attention_masks = encode_batch(texts, MAX_LEN)
labels = torch.tensor(labels)

print("✅ Đã mã hóa xong!")

🤖 Đang tải Tokenizer từ VinAI...
🚀 Bắt đầu mã hóa (Tokenizing)...
🚀 Bắt đầu mã hóa (Tokenizing)...


100%|██████████| 96239/96239 [03:37<00:00, 442.66it/s]


✅ Đã mã hóa xong!


# Chia tập Train / Val & Tạo DataLoader

In [5]:
# Chia train (85%) - validation (15%)
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, random_state=42, test_size=0.15, stratify=labels
)
train_masks, val_masks, _, _ = train_test_split(
    attention_masks, labels, random_state=42, test_size=0.15, stratify=labels
)

# Tạo DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

print(f"🔹 Train set: {len(train_inputs)}")
print(f"🔹 Val set:   {len(val_inputs)}")

🔹 Train set: 81803
🔹 Val set:   14436


# Định nghĩa Model PhoBERT Classifier

In [6]:
class PhoBertClassifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(PhoBertClassifier, self).__init__()
        # Load PhoBERT gốc
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        
        # Nếu muốn train nhanh hơn thì đóng băng BERT (freeze_bert=True)
        # Nhưng để độ chính xác cao nhất thì nên để False (Fine-tune toàn bộ)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
                
        # Lớp phân loại tùy chỉnh
        self.classifier = nn.Sequential(
            nn.Linear(768, 512), # 768 là hidden size của PhoBERT base
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 2)    # Output 2 lớp: Fake (0) và Real (1)
        )

    def forward(self, input_ids, attention_mask):
        # Forward qua BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Lấy vector [CLS] đại diện cho cả câu
        cls_output = outputs.last_hidden_state[:, 0, :]
        
        # Forward qua lớp phân loại
        logits = self.classifier(cls_output)
        return logits

# Khởi tạo model
model = PhoBertClassifier(freeze_bert=False)
model.to(device) # Đưa model vào GPU

# Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss()

print("✅ Đã khởi tạo Model thành công.")

✅ Đã khởi tạo Model thành công.


# Bắt đầu Training

In [ ]:
import random

# Hàm tính độ chính xác
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_val)

# Biến lưu lịch sử để vẽ biểu đồ
training_stats = []

print("🚀 BẮT ĐẦU TRAINING...")

for epoch_i in range(0, EPOCHS):
    print(f'\n======== Epoch {epoch_i + 1} / {EPOCHS} ========')
    print('Training...')
    
    t0 = time.time()
    total_train_loss = 0
    model.train()

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch_i+1}")

    for step, batch in enumerate(progress_bar):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        logits = model(b_input_ids, b_input_mask)
        loss = loss_fn(logits, b_labels)
        
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'loss': loss.item()})

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = time.time() - t0
    
    # ================= VALIDATION =================
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    
    total_eval_accuracy = 0
    total_eval_loss = 0
    
    # Lưu kết quả để vẽ Confusion Matrix sau này
    val_preds = []
    val_true = []

    for batch in val_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():
            logits = model(b_input_ids, b_input_mask)
            loss = loss_fn(logits, b_labels)
            
        total_eval_loss += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
        # Lưu lại dự đoán
        val_preds.extend(np.argmax(logits, axis=1).flatten())
        val_true.extend(label_ids.flatten())

    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    avg_val_loss = total_eval_loss / len(val_dataloader)
    val_time = time.time() - t0
    
    print(f"  ✅ Avg Train Loss: {avg_train_loss:.4f}")
    print(f"  ✅ Avg Val Loss:   {avg_val_loss:.4f}")
    print(f"  ✅ Val Accuracy:   {avg_val_accuracy:.4f}")

    # LƯU LẠI LỊCH SỬ (QUAN TRỌNG)
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': val_time
        }
    )

print("\n🏁 TRAINING COMPLETE!")

🚀 BẮT ĐẦU TRAINING...

======== Epoch 1 / 3 ========
Training...


Epoch 1:   0%|          | 0/5113 [00:00<?, ?it/s]


KeyboardInterrupt: 

# Loss & Accuracy

In [ ]:
# --- CELL MỚI: VẼ BIỂU ĐỒ ---
import matplotlib.pyplot as plt
import seaborn as sns

# Tạo DataFrame từ lịch sử training
df_stats = pd.DataFrame(data=training_stats)

# Sử dụng style seaborn cho đẹp
sns.set(style='darkgrid')

# Tăng kích thước phông chữ và hình
plt.rcParams["figure.figsize"] = (12,6)

# Vẽ biểu đồ 1: Learning Curve (Loss)
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([i for i in range(len(df_stats))], df_stats['epoch']) # Set trục x là số epoch

plt.show()

# Vẽ biểu đồ 2: Accuracy
plt.figure()
plt.plot(df_stats['Valid. Accur.'], 'r-o', label="Accuracy")
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.xticks([i for i in range(len(df_stats))], df_stats['epoch'])

plt.show()

# Confusion Matrix

In [ ]:
# --- CELL MỚI: VẼ CONFUSION MATRIX ---
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Tính toán ma trận nhầm lẫn dựa trên Epoch cuối cùng
cm = confusion_matrix(val_true, val_preds)

# Vẽ Heatmap
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Fake', 'Real'], 
            yticklabels=['Fake', 'Real'])

plt.ylabel('Thực tế (Ground Truth)')
plt.xlabel('Dự đoán (Prediction)')
plt.title('Confusion Matrix - Kết quả phân loại')
plt.show()

# In lại báo cáo chi tiết
print(classification_report(val_true, val_preds, target_names=['Fake', 'Real']))

# Lưu Model & Test thử

In [ ]:
def predict_news_logic(text, threshold=0.80): # Ngưỡng tin cậy 80%
    # 1. Chuẩn bị dữ liệu
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding='max_length')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 2. Model dự đoán
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        probs = torch.nn.functional.softmax(logits, dim=1)
        
    fake_prob = probs[0][0].item() # Xác suất là Fake (Label 0)
    real_prob = probs[0][1].item() # Xác suất là Real (Label 1)
    
    print(f"\n📰 Input: {text[:50]}...")
    print(f"📊 Chỉ số: Real={real_prob:.2f} | Fake={fake_prob:.2f}")

    # 3. LOGIC QUYẾT ĐỊNH (Quan trọng nhất)
    # Nếu chắc chắn là Real (> 80%)
    if real_prob > threshold:
        return 1  # Label REAL
        
    # Nếu chắc chắn là Fake (> 80%)
    elif fake_prob > threshold:
        return 0  # Label FAKE
        
    # Nếu lấp lửng (không bên nào > 80%)
    else:
        return 2  # Label UNDEFINED (Hệ thống tự gán)

# --- TEST THỬ ---
# Case 1: Rất rõ ràng
kq1 = predict_news_logic("Bộ Y tế chính thức phê duyệt vắc xin mới.") 
print(f"👉 Kết quả hệ thống trả về: {kq1}") 

# Case 2: Mơ hồ / Bịa đặt lạ lẫm
kq2 = predict_news_logic("Người ngoài hành tinh đang ăn bún đậu mắm tôm.")
print(f"👉 Kết quả hệ thống trả về: {kq2}")